# LSTM for Predicting PV energy generation   

In [9]:
# Data handling and manipulation
import pandas as pd
import numpy as np

# Machine learning and preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Deep learning (LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np
# Plotting and visualization
import matplotlib.pyplot as plt

In [10]:
# Step 1: Load the dataset
file_path = 'solar_data_with_temperature_adjusted_irradiance.csv'
data = pd.read_csv(
    file_path,
    index_col=[0],  # Use 'time' as the index
    parse_dates=[0]  # Parse the 'time' column as datetime
).drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'])


In [11]:
split = data.index[int(len(data) * 0.7)]

train = data.loc[data.index <= split].copy()
test = data.loc[data.index > split].copy()


In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Step 2: Separate the features (X) and the target (y)
def create_features(data, label=None):
    X = data[['GHI', 'DNI', 'DHI', 'Temperature']]  # Features
    if label:
        y = data[label]  # Target variable
        return X, y
    return X

X_train, y_train = create_features(train, label='Tilted Irradiance (Adjusted)')
X_test, y_test = create_features(test, label='Tilted Irradiance (Adjusted)')

# Step 6: Apply ColumnTransformer for preprocessing to the dataset
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['GHI', 'DNI', 'DHI', 'Temperature']),  # Scale numerical features
    ]
)

# Scale the data (apply transformation on both training and testing data)
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

# Reshape the data for LSTM (3D array)
# For LSTM, we need a 3D shape (samples, time_steps, features), with time_steps = 1 for each sample in this case.
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])  # 1 timestep
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])  # 1 timestep



In [13]:
# Step 6: Define the LSTM model
def lstm_model(X_train_scaled, y_train):
    """
    Builds and trains an LSTM model using the training data.

    Parameters:
        X_train_scaled (np.ndarray): Scaled input training data.
        y_train (np.ndarray): Target training data.

    Returns:
        Sequential: Trained LSTM model.
    """
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=60, activation='relu', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(units=80, activation='relu', return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(units=120, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1))  # Output layer for single prediction
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=0)  # Train with verbose=0 to suppress output
    return model

# Step 7: Apply K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
mean_errors = []
std_errors = []

# Track the MSE for each fold
for train_idx, val_idx in kf.split(X_train_scaled):
    X_train_cv, X_val_cv = X_train_scaled[train_idx], X_train_scaled[val_idx]
    y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]
    
    # Train LSTM model
    model = lstm_model(X_train_cv, y_train_cv)
    
    # Make predictions on the validation fold
    y_pred = model.predict(X_val_cv)
    
    # Calculate Mean Squared Error for this fold
    mse = mean_squared_error(y_val_cv, y_pred)
    mean_errors.append(mse)

# Calculate mean error (MSE) and variance (standard deviation) across all folds
mean_mse = np.mean(mean_errors)
std_mse = np.std(mean_errors)

print(f"Mean MSE across all folds: {mean_mse:.4f}")
print(f"Variance (Standard Deviation) of MSE across folds: {std_mse:.4f}")

C:\Users\giuli\AppData\Local\Temp\ipykernel_29256\35471574.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]
c:\Users\giuli\Repositories\aee_2425\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


C:\Users\giuli\AppData\Local\Temp\ipykernel_29256\35471574.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]
c:\Users\giuli\Repositories\aee_2425\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


C:\Users\giuli\AppData\Local\Temp\ipykernel_29256\35471574.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]
c:\Users\giuli\Repositories\aee_2425\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


C:\Users\giuli\AppData\Local\Temp\ipykernel_29256\35471574.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]
c:\Users\giuli\Repositories\aee_2425\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


C:\Users\giuli\AppData\Local\Temp\ipykernel_29256\35471574.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]
c:\Users\giuli\Repositories\aee_2425\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Mean MSE across all folds: 359.8757
Variance (Standard Deviation) of MSE across folds: 195.1798


In [14]:
all = pd.concat([test, train], sort = False)

In [15]:
_ = all[['Tilted Irradiance (Adjusted)', 'Prediction']].plot(figsize=(15,15))




KeyError: "['Prediction'] not in index"

In [ ]:
# Step 1: Calculate the threshold for the top 10% of 'Tilted Irradiance (Adjusted)'
top_10_percent_threshold = all['Tilted Irradiance (Adjusted)'].quantile(0.70)

# Step 2: Filter the data to only include the top 10% based on 'Tilted Irradiance (Adjusted)'
top_10_percent_data = all[all['Tilted Irradiance (Adjusted)'] >= top_10_percent_threshold]

# Step 3: Plot the filtered data as a line plot for actual and predicted data
plt.figure(figsize=(12, 6))

# Plot the actual data (blue)
plt.plot(top_10_percent_data.index, top_10_percent_data['Tilted Irradiance (Adjusted)'], label='Actual', color='blue')

# Plot the predicted data (red)
plt.plot(top_10_percent_data.index, top_10_percent_data['Prediction'], label='Prediction', color='red')

# Optional: Add titles, labels, and a legend
plt.title("Top 30% of 'Tilted Irradiance (Adjusted)' vs Prediction")
plt.xlabel("Datetime")  # or "Index" if the index is not datetime
plt.ylabel("Irradiance / Prediction")
plt.legend()

# Rotate x-axis labels if necessary for readability
plt.xticks(rotation=45)

# Show the plot with tight layout for spacing
plt.tight_layout()
plt.show()


In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)

# Plot actual vs predicted MW values, with distinct line styles
_ = all[['Prediction', 'Tilted Irradiance (Adjusted)']].plot(ax=ax,
                                                style=['-', '.'],
                                                color=['red', 'blue'])  # Red for Prediction, Blue for Actuals

# Set x-axis bounds to zoom in on January 2015
ax.set_xlim(pd.to_datetime('2019-11-01'), pd.to_datetime('2019-12-01'))

# Set y-axis limits to match the range from 0 to 60000
#ax.set_ylim(0, 60000)

# Add a title for the plot
plt.suptitle('December 2019 Forecast vs Actuals')

# Optional: Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


In [16]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred, handling zero values in y_true."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Mask where y_true is zero or NaN
    valid_mask = (y_true != 0) & ~np.isnan(y_true)
    
    # Apply the mask to both y_true and y_pred
    y_true_valid = y_true[valid_mask]
    y_pred_valid = y_pred[valid_mask]
    
    # If there are no valid points, return NaN or some other indicator (e.g., inf)
    if len(y_true_valid) == 0:
        return np.nan
    
    # Calculate the MAPE only on valid (non-zero, non-NaN) values
    mape = np.mean(np.abs((y_true_valid - y_pred_valid) / y_true_valid)) * 100
    
    return mape

# Example usage:
mape_value = mean_absolute_percentage_error(y_true=test['Tilted Irradiance (Adjusted)'], y_pred=test['Prediction'])
print(f"MAPE: {mape_value:.2f}%")

# Your true and predicted values
y_true = test['Tilted Irradiance (Adjusted)']
y_pred = test['Prediction']

# Ensure that both y_true and y_pred are arrays or series of the same length
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Mean Squared Error (MSE)
mse = mean_squared_error(y_true, y_pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# R-squared (R²) - how well the model explains the variance
r2 = r2_score(y_true, y_pred)
print(f"R-squared (R²): {r2:.2f}")



KeyError: 'Prediction'


# Model Evaluation and Conclusions

## Model Performance Metrics:

The evaluation of the model was done using several performance metrics:

- **Mean Absolute Percentage Error (MAPE)**: 16.21%
- **Mean Absolute Error (MAE)**: 10.18
- **Mean Squared Error (MSE)**: 385.89
- **Root Mean Squared Error (RMSE)**: 19.64
- **R-squared (R²)**: 0.99

## Interpretation of Results:

1. **MAPE (16.21%)**:
   - The Mean Absolute Percentage Error (MAPE) indicates that the model's predictions deviate from the actual values by approximately 16.21% on average. A MAPE value below 20% is typically considered good in predictive modeling, particularly for time-series forecasts. This suggests the model is performing reasonably well.

2. **MAE (10.18)**:
   - The Mean Absolute Error (MAE) measures the average magnitude of the errors in a set of predictions, without considering their direction. An MAE of 10.18 means that, on average, the model's predictions are off by around 10.18 units (MW) from the actual values.

3. **MSE (385.89) & RMSE (19.64)**:
   - The Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) provide a sense of the magnitude of the errors, with RMSE being particularly useful for understanding the scale of the error in the original units. The relatively low RMSE (19.64) indicates that, while there are errors, they are not large in the context of the problem.

4. **R-squared (0.99)**:
   - The R-squared value of 0.99 indicates that the model is able to explain 99% of the variance in the target variable, which is excellent. A high R² value suggests that the model fits the data very well and is likely capturing the underlying patterns in the time series accurately.

## Conclusion:

The model performs very well, with a high R-squared value, indicating that it can explain most of the variance in the data. The MAPE value of 16.21% is reasonable, suggesting that the model has a good predictive performance, though there is still room for improvement. The MAE, MSE, and RMSE values suggest that the model's predictions are fairly close to the actual values, with relatively small errors. 

Overall, the LSTM model appears to be a strong fit for predicting the solar energy generation based on the provided features.
